In [1]:
import io
import csv
import os.path

import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt

%matplotlib qt5

In [2]:
from posarmctools.ekfnavtools import *
%load_ext autoreload
%autoreload 2

In [3]:
blocksPerFile = 6
samplesPerRamp = 6000
rampsPerFile = 1500
samplesPerFile = 9000000
T_files = samplesPerFile / 10e6

## Load navigation data SbgLogEkfNav

In [11]:
prefix = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/dataLogger/session_0004/2017_10_12/11h00/"
prefix_2 = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/dataLogger/session_0004/2017_10_12/12h00/"

logEkfEuler = prefix + "sbgLogEkfEuler.dat"
logEkfNav  = prefix + "sbgLogEkfNav.dat"
logUtcData = prefix + "sbgLogUtcData.dat"

logEkfEuler_2 = prefix_2 + "sbgLogEkfEuler.dat"
logEkfNav_2  = prefix_2 + "sbgLogEkfNav.dat"
logUtcData_2 = prefix_2 + "sbgLogUtcData.dat"

In [14]:
# logEkfNav 11h00
timeStamps_1 = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_timeStamp )
Long_1 = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Long )
Lat_1 = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Lat )
Alt_1 = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Alt )
vN_1, vE_1, vD_1 = np.loadtxt( logEkfNav, skiprows = 1, usecols = (idx_velNorth, idx_velEast, idx_velDown), unpack=True )

In [12]:
# logEkfNav 12h00
timeStamps_2 = np.loadtxt( logEkfNav_2, skiprows = 1, usecols = idx_timeStamp )
Long_2 = np.loadtxt( logEkfNav_2, skiprows = 1, usecols = idx_Long )
Lat_2 = np.loadtxt( logEkfNav_2, skiprows = 1, usecols = idx_Lat )
Alt_2 = np.loadtxt( logEkfNav_2, skiprows = 1, usecols = idx_Alt )
vN_2, vE_2, vD_2 = np.loadtxt( logEkfNav, skiprows = 1, usecols = (idx_velNorth, idx_velEast, idx_velDown), unpack=True )

In [15]:
Long = np.concatenate((Long_1, Long_2))
Lat = np.concatenate((Lat_1, Lat_2))
Alt = np.concatenate((Alt_1, Alt_2))
timeStamps = np.concatenate((timeStamps_1, timeStamps_2))

In [16]:
Vel_1 = (vN_1**2 + vE_1**2 + vD_1**2)**0.5
Vel_2 = (vN_2**2 + vE_2**2 + vD_2**2)**0.5

In [17]:
Vel = np.concatenate((Vel_1, Vel_2))

### Plot navigation data jointly with track references

In [18]:
plt.figure()
ax = plt.subplot(111)
plotLongLatAndTrackReferences( ax, Long, Lat )
plt.title( "Navigation data and track references\n" + prefix )
plt.grid()

In [19]:
validNav = np.where(Long < 0)

In [20]:
plt.figure()
ax = plt.subplot(111)
plotLongLatAndTrackReferences( ax, Long[validNav], Lat[validNav] )
plt.title( "Navigation data and track references\n" + prefix )
plt.grid()

## Look for the time data corresponding to the records

In [21]:
hourOffset = 2

#timeStamp status year month day hour minute second nanoSecond gpsTimeOfWeek
utc_timeStamp_1 = np.loadtxt( logUtcData, skiprows = 1, usecols = 0, unpack=True )

In [63]:
#timeStamp status year month day hour minute second nanoSecond gpsTimeOfWeek
utc_timeStamp_2 = np.loadtxt( logUtcData_2, skiprows = 1, usecols = 0, unpack=True )

In [64]:
utc_timeStamp = np.concatenate((utc_timeStamp_1, utc_timeStamp_2))

### Process timeStamps.data

In [72]:
data_dir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/2017_10_12/2017_10_12_13_58_36"
timeStampsFile = data_dir + "/2017_10_12_13_58_36_timeStamps.data"
blockNumber_1, timeStamp_1 = np.loadtxt( timeStampsFile, skiprows = 1, unpack=True )

In [73]:
data_dir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/2017_10_12/2017_10_12_14_04_34"
timeStampsFile = data_dir + "/2017_10_12_14_04_34_timeStamps.data"
blockNumber_2, timeStamp_2 = np.loadtxt( timeStampsFile, skiprows = 1, unpack=True )

In [74]:
data_dir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/2017_10_12/2017_10_12_14_17_27"
timeStampsFile = data_dir + "/2017_10_12_14_17_27_timeStamps.data"
blockNumber_3, timeStamp_3 = np.loadtxt( timeStampsFile, skiprows = 1, unpack=True )

In [75]:
timeStampStart_1 = timeStamp_1[0]
timeStampStop_1 = timeStamp_1[-1]

timeStampStart_2 = timeStamp_2[0]
timeStampStop_2 = timeStamp_2[-1]

timeStampStart_3 = timeStamp_3[0]
timeStampStop_3 = timeStamp_3[-1]

In [76]:
idxStart = np.where(utc_timeStamp <= timeStampStart_1)[0][-1]
idxStop = np.where(utc_timeStamp <= timeStampStop_1)[0][-1]
record1_idx_alt = (idxStart, idxStop)
print( "idxStart = {}, idxStop = {}".format(idxStart, idxStop) )

idxStart = 184526, idxStop = 197306


In [77]:
idxStart = np.where(utc_timeStamp <= timeStampStart_2)[0][-1]
idxStop = np.where(utc_timeStamp <= timeStampStop_2)[0][-1]
record2_idx_alt = (idxStart, idxStop)
print( "idxStart = {}, idxStop = {}".format(idxStart, idxStop) )

idxStart = 256130, idxStop = 271070


In [78]:
idxStart = np.where(utc_timeStamp <= timeStampStart_3)[0][-1]
idxStop = np.where(utc_timeStamp <= timeStampStop_3)[0][-1]
record3_idx_alt = (idxStart, idxStop)
print( "idxStart = {}, idxStop = {}".format(idxStart, idxStop) )

idxStart = 410887, idxStop = 426187


In [79]:
plt.plot(timeStamps)

### Plot navigation data and record periods

In [81]:
plt.figure()

ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax, Long[validNav], Lat[validNav] )
plotRunaway( ax )

addOnPlot( ax, Long, Lat, record1_idx_alt, 'orange', 'record 13_58_36 (time line dated by SBG time stamps)' )
addOnPlot( ax, Long, Lat, record2_idx_alt, 'red', 'record 14_04_34 (time line dated by SBG time stamps)' )
addOnPlot( ax, Long, Lat, record3_idx_alt, 'green', 'record 14_17_27 (time line dated by SBG time stamps)' )

plt.title("Navigation data, track references, record locations\nFlight 2017_10_12")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.grid()

## Plot attitude data

In [ ]:
idx_timeStamp = 0
idx_roll = 1
idx_pitch = 2
idx_yaw = 3
idx_rollStdDev = 4
idx_pitchStdDev = 5
idx_yawStdDev = 6
idx_status = 7

In [ ]:
euler_timeStamp, roll, pitch, yaw = np.loadtxt( logEkfEuler, skiprows = 1, 
                                               usecols = (0, idx_roll, idx_pitch, idx_yaw), unpack=True )

### Track 1

In [ ]:
idxRange = range(record1_idx_alt[0], record1_idx_alt[1])

plt.figure()

plt.subplot(211)
plt.title("Altitude and velocity during record record 13_37_22")
plt.plot( Alt[idxRange], label="altitude" )
plt.legend()
plt.grid()
plt.subplot(212)
plt.plot( Vel[idxRange], label="velocity" )
plt.legend()
plt.grid()

In [ ]:
plt.figure()

plt.subplot(311)
plt.title("Pitch roll and yaw during record record 13_37_22")
plt.plot( pitch[idxRange], label="pitch" )
plt.legend()
plt.grid()

plt.subplot(312)
plt.plot( roll[idxRange], label="roll" )
plt.legend()
plt.grid()

plt.subplot(313)
plt.plot( yaw[idxRange], label="yaw" )
plt.legend()
plt.grid()